In [1]:
import tensorflow as tf
from tensorflow import keras
import mnist_reader
import numpy as np
import os


In [2]:
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')
learning_rate = 0.001
training_epochs = 100
batch_size = 64
display_step = 20


In [3]:
def AlexNet():
    inputs = keras.Input(shape=(28,28,1))
    
    conv1 = keras.layers.Conv2D(filters=64,kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D(pool_size=(2,2),padding='same')(conv1)
    norm1 = keras.layers.BatchNormalization()(pool1)
    drop1 = keras.layers.Dropout(0.5)(norm1)
    
    conv2 = keras.layers.Conv2D(filters=128,kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.relu)(drop1)
    pool2 = keras.layers.MaxPool2D(pool_size=(2,2),padding='same')(conv2)
    norm2 = keras.layers.BatchNormalization()(pool2)
    drop2 = keras.layers.Dropout(0.5)(norm2)

    conv3 = keras.layers.Conv2D(filters=256,kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.relu)(drop2)
    pool3 = keras.layers.MaxPool2D(pool_size=(2,2),padding='same')(conv3)
    norm3 = keras.layers.BatchNormalization()(pool3)
    drop3 = keras.layers.Dropout(0.5)(norm3)
    
    flat = keras.layers.Flatten()(drop3)
    dense1 = keras.layers.Dense(units=1024, activation=tf.nn.relu)(flat)
    dense2 = keras.layers.Dense(units=1024, activation=tf.nn.relu)(dense1)
    
    logits = keras.layers.Dense(units=10,activation='softmax')(dense2)
    return keras.Model(inputs=inputs, outputs=logits)

In [4]:
model = AlexNet()
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     4,195,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,626,634 (21.46 MB)

 Trainable params: 5,625,738 (21.46 MB)

 Non-trainable params: 896 (3.50 KB)

In [5]:
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, 10)  # shape: (60000, 10)
y_test  = to_categorical(y_test, 10)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=training_epochs,
    validation_split=0.1,
    verbose=2  # display_step 대체
)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ 테스트 정확도: {test_acc:.4f}")

Epoch 1/100
844/844 - 49s - 58ms/step - accuracy: 0.8146 - loss: 0.5103 - val_accuracy: 0.8560 - val_loss: 0.3948
Epoch 2/100
844/844 - 46s - 54ms/step - accuracy: 0.8787 - loss: 0.3314 - val_accuracy: 0.8885 - val_loss: 0.3075
Epoch 3/100
844/844 - 46s - 54ms/step - accuracy: 0.8871 - loss: 0.3013 - val_accuracy: 0.8932 - val_loss: 0.2899
Epoch 4/100


KeyboardInterrupt: 